In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import numpy as np
import matplotlib.pyplot as plt

import time

In [31]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')

        self.layers = layers
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

        self.H1 = self.linears[0]

        
    'forward pass'
    def forward(self,x,t):              
        
        # for i in range(len(self.layers)-2):
        #     z = self.linears[i](a)
        #     a = self.activation(z)

        a = torch.cat([x,t], dim = 1)    #(10000,2)

        for i in range(len(self.layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)


            
        b = self.linears[-1](a) 
         
        return b
    
    def forward_direct(self, x):
        
        z = x.float()
        H = self.linears[0].weight

        for i in range(len(self.layers)-2):
            L = self.linears[i](z)
            z = self.activation(L)
            G = (1-torch.square(z))*H.t() #\sigma'(L)*H
            H = torch.matmul(self.linears[i+1].weight,G.t())

        z = self.linears[-1](z)
         
        return z,H

In [32]:
layers = np.array([2,50,50,50,1])
PINN = Sequentialmodel(layers)

In [47]:
# Create the training data

x = torch.linspace(-1,1,100, requires_grad = True).view(-1,1)
t = torch.linspace(0,1,100, requires_grad = True).view(-1,1)


if torch.is_tensor(x) != True:         
    x = torch.from_numpy(x)  
if torch.is_tensor(t) != True:         
    t = torch.from_numpy(t) 

#convert to float
x = x.float()
t = t.float()

    
x_train,t_train = torch.meshgrid(x.squeeze(),t.squeeze(), indexing = 'xy')
x_train = x_train.reshape(-1,1).requires_grad_(True)
t_train = t_train.reshape(-1,1).requires_grad_(True)



In [48]:
def pde_residual(x, t, nu):
    u = PINN(x,t)

    du_dx = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    du_dt = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]
    du_dx_x = torch.autograd.grad(du_dx, x, torch.ones_like(du_dx), create_graph=True)[0]

    res_pde = du_dt - (nu / (np.pi)) * du_dx_x + u * du_dx

    return res_pde


    

In [49]:

def initial_condition(x):
  u_ic = PINN(x, torch.zeros_like(x))
  res_ic = u_ic - (-torch.sin(np.pi * x))
  return res_ic

In [50]:
def boundary_condition(t):
    u_left = PINN(torch.full_like(t, -1), t)
    u_right = PINN(torch.full_like(t, 1), t)

    res_left = u_left - torch.zeros_like(t)
    res_right = u_right - torch.zeros_like(t)

    return res_left, res_right

In [51]:
def compute_losses():
   res_pde = pde_residual(x_train, t_train, nu = 0.01) 
   res_ic = initial_condition(x_train)
   res_left, res_right = boundary_condition(t_train)

   loss_pde = torch.mean(res_pde**2)
   loss_ic = torch.mean(res_ic**2)
   loss_bc = torch.mean(res_left**2) + torch.mean(res_right**2)

   total_loss = loss_pde + loss_ic + loss_bc

   return total_loss



In [52]:
optimizer = torch.optim.Adam(PINN.parameters(), lr=0.01)

In [54]:
start_time = time.time()

num_epochs = 5000



for epoch in range(num_epochs):
    optimizer.zero_grad()

    total_loss = compute_losses()

    
    total_loss.backward()

    optimizer.step()

    if (epoch) % 500 == 0:
     print(f'Epoch {epoch}, Loss: {total_loss.item()}')


    








Epoch 0, Loss: 0.05367458239197731
Epoch 500, Loss: 0.05935884639620781
Epoch 1000, Loss: 0.06171201169490814
Epoch 1500, Loss: 0.04555651918053627
Epoch 2000, Loss: 0.043755561113357544
Epoch 2500, Loss: 0.021487591788172722
Epoch 3000, Loss: 0.10012179613113403
Epoch 3500, Loss: 0.08944450318813324
Epoch 4000, Loss: 0.059110794216394424
Epoch 4500, Loss: 0.09762782603502274
